# MNIST-Streamlit tutorial

Note: This notebook runs on Python 3.8 and uses UbiOps Client Library 3.15.0.

In this notebook we will show you the following:

How to connect a deployment with streamlit to build a web app.

The deployment in this notebook is based on the [image recognition](https://ubiops.com/docs/ubiops_tutorials/ready-deployments/image-recognition/image-recognition/) ready deployment

If you run this entire notebook after filling in your acces token, the MNIST deployment will be deployed to your UbiOps environment. If you want, you can go to your environment and check out your deployment there first. After that you can initialize the mnist_streamlit.py and check out a web app version of the deployment.

We recommend to run the cells step by step, as some cells can take a few minutes to finish. You can run everything in one go as well and it will work, just allow a few minutes for building the deployment.

## Establishing a connection with your UbiOps environment

### Add your API token and project name to the first and third cell block. We provide a deployment and deployment version name. Afterwards we initialize the client library. This way we can deploy the MNIST model to your environment.

In [ ]:
API_TOKEN = '<INSERT API_TOKEN WITH PROJECT EDITOR RIGHTS>' # Make sure this is in the format "Token token-code"
PROJECT_NAME = '<INSERT PROJECT NAME IN YOUR ACCOUNT>'

DEPLOYMENT_NAME = 'mnist-streamlit'
DEPLOYMENT_VERSION = 'v1'

#import all necessary libraries:
import os
import ubiops

client = ubiops.ApiClient(ubiops.Configuration(
    api_key={'Authorization': API_TOKEN}, 
    host="https://api.ubiops.com/v2.1"
))

api = ubiops.CoreApi(client)

## Download the MNIST image recognition model 

You can download the model [here](https://download-github.ubiops.com/?_gl=1*ql909b*_ga*MTEzMDc3Nzg3Mi4xNjMyNDkyODk3*_ga_S22LQ1P875*MTYzNjQ0ODMzMy4yNjUuMS4xNjM2NDUwMzEyLjA.#!/home?url=https:%2F%2Fgithub.com%2FUbiOps%2Ftutorials%2Ftree%2Fmaster%2Fready-deployments%2Fimage-recognition%2Fmnist_deployment_package)

## Create the deployment

In [ ]:
# Create the deployment
deployment_template = ubiops.DeploymentCreate(
    name = DEPLOYMENT_NAME,
    description = 'image-recognition',
    input_type = 'structured',
    output_type = 'structured',
    input_fields = [
        {'name' : 'image', 'data_type' : 'file'},
    ],
    output_fields = [
        {'name': 'prediction', 'data_type': 'int'},
        {'name': 'probability', 'data_type': 'double'}
    ],
    labels= {'demo': 'MNIST-Streamlit'}
)

api.deployments_create(
    project_name=PROJECT_NAME,
    data=deployment_template
)

# Create the version
version_template = ubiops.DeploymentVersionCreate(
    version=DEPLOYMENT_VERSION,
    environment='python3-8',
    instance_type='512mb',
    minimum_instances=0,
    maximum_instances=1,
    maximum_idle_time=1800, # = 30 minutes
    request_retention_mode='none' # we don't need request storage in this example
)

api.deployment_versions_create(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    data=version_template
)

# Upload the zipped deployment package
file_upload_result =api.revisions_file_upload(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    version=DEPLOYMENT_VERSION,
    file='mnist_deployment_package.zip'
)

In [ ]:
ubiops.utils.wait_for_deployment_version(
    client=api.api_client,
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    version=DEPLOYMENT_VERSION,
    revision_id=file_upload_result.revision
)

## Enter your API token and Project in the Streamlit file

In [ ]:
%%writefile mnist-streamlit.py
import streamlit as st
import ubiops 
import tempfile
from time import sleep

st.title("Streamlit and UbiOps example")

# Connect with your UbiOps environment
API_TOKEN = '<INSERT API_TOKEN WITH PROJECT EDITOR RIGHTS>' # Make sure this is in the format "Token token-code"
PROJECT_NAME = '<INSERT PROJECT NAME IN YOUR ACCOUNT>'

DEPLOYMENT_NAME = 'mnist-streamlit'

# API setup 
if PROJECT_NAME and API_TOKEN and DEPLOYMENT_NAME:
    # Only reconnect if API object is not in session state
    if 'ubiops_api' not in st.session_state:
        with st.spinner("Connecting to UbiOps API"):
            configuration = ubiops.Configuration(host="https://api.ubiops.com/v2.1")
            configuration.api_key['Authorization'] = API_TOKEN

            st.session_state.client = ubiops.ApiClient(configuration)
            st.session_state.ubiops_api = ubiops.CoreApi(st.session_state.client)
            deployment_info = st.session_state.ubiops_api.deployments_get(PROJECT_NAME,DEPLOYMENT_NAME)
           
            print(deployment_info)
            
            sleep(2) # sleep for 2s to showcase progress spinners
            
            # Use the streamlit session to store API object
            if(st.session_state.ubiops_api.service_status().status == 'ok' ):
                st.success("Connected to UbiOps API!")
            else:
                st.error("Not connected!")
                


# File upload
upload_file = st.file_uploader("Choose a file")
if upload_file is not None:
    if 'results' not in st.session_state:
        st.session_state.results = []
    with open("out.txt", "wb") as outfile:
        # Copy the BytesIO stream to the output file
        outfile.write(upload_file.getvalue())
    file_uri = ubiops.utils.upload_file(st.session_state.client, PROJECT_NAME, 'out.txt')
    # Make a request using the file URI as input.
    data = {'image': file_uri}
    
    result = st.session_state.ubiops_api.deployment_requests_create(
        project_name=PROJECT_NAME,
        deployment_name=DEPLOYMENT_NAME,
        data=data
    )
    # Store results in session
    st.session_state.results.append([result,upload_file])

# Show all results in from session
if 'results' in st.session_state: 
    for r in st.session_state.results[::-1]:
        c1, c2 = st.columns(2)
        c2.write(r[0].result)
        c1.image(r[1])

## Install the streamlit package

In [ ]:
!pip install streamlit

## Now it is time to connect to streamlit and turn the deployment we have just created into an app

In [ ]:
!streamlit run mnist-streamlit.py